In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt

mnist=input_data.read_data_sets('MNIST_data',one_hot=True) #Auto download images from Web sites

#add one more layer and return te output of this layer
#======================================================================
#def add_layer(inputs,in_size,out_size,activation_function=None):
    #Weights=tf.Variable(tf.random_normal([in_size,out_size]))   #inital vaule prefer to be non-zero
              #print(Weights)
   # biases=tf.Variable(tf.zeros([1,out_size])+0.1)  # size= 1*out_size, all values are 0.1(non-Zero)

    #Wx_plus_b=tf.matmul(inputs,Weights)+biases # The unactiviated values are storaged in Wx_plus_b
    # name of activiation funtion is default 
    
    #if activation_function is None:  # None means 0, activation fucntion is linear
       # outputs=Wx_plus_b # keep original value
   # else:
       # outputs=activation_function(Wx_plus_b)
    #return outputs

#=====================================================================
#Define compute_accuracy
def compute_accuracy(v_xs,v_ys):
    global predition  #global variable
    y_pre=sess.run(predition,feed_dict={xs:v_xs,keep_prob:1})  #prediction value.y_pre is a 10*1 matrix, the values in it are not integer but float, they means probability for each number0~9
    correct_predition=tf.equal(tf.argmax(y_pre,1),tf.argmax(v_ys,1)) #Is the max value in ys equal to the max one in y_pre
    accuracy=tf.reduce_mean(tf.cast(correct_predition,tf.float32)) #tf.cast?????
    result=sess.run(accuracy,feed_dict={xs:v_xs,ys:v_ys,keep_prob:1})
    return result

#======================================================================
#Define variable

def weight_variable(shape):
    initial=tf.truncated_normal(shape,stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial=tf.constant(0.1,shape=shape) #bias generally is positive.Here is 0.1 for all positions
    return tf.Variable(initial)
#=====================================================================
#=====================================================================
#Define CNN methods: Same padding, maxpooling
#strides[1,x_movement,y_movement,1]
#Must stride[0],stride[3]=1
def conv2d(x,W):  #x: input value  W:weight
    return tf.nn.conv2d(x,W,strides=[1,1,1,1],padding='SAME')   #s4 trides: 1*matrix, first and last values are 1. 2nd 1 means stride of x-axis is 1, 3rd 1 means stride of y-axis is 1
    

def max_pool_2x2(x):
    return tf.nn.max_pool(x,ksize=[1,2,2,1],strides=[1,2,2,1],padding='SAME') #K-size is kernal size

#=====================================================================================
#generate input trained data and real data
#None for this exmaple

#=========================================================================================
#define placeholder for inputs to network
xs=tf.placeholder(tf.float32,[None,784])/255  #None means any quantity, 784(28*28) means the pixel size of each image
#/255 means normalized and dtype from integer becomes to float32
#xs=tf.placeholder(tf.float32,[None,784])
#print(xs.shape)
ys=tf.placeholder(tf.float32,[None,10])  
#print(ys.shape)
keep_prob=tf.placeholder(tf.float32)
x_image=tf.reshape(xs,[-1,28,28,1])#xs:input data. -1: don't care xs's orginal dimension. 28*28. 1: the color is black/while, the channel is 1
#print(x_image.shape)  #[n_samples,28,28,1]
#print(x_image)
#======================================================================
#Define CNN layer
#conv1 layer + Pooling########
W_conv1=weight_variable([5,5,1,32]) #patch(filter) side:5x5, 1:in side(unit or depth per patch), 32:out side(output depth= # of patch(filter))
b_conv1=bias_variable([32]) #the max length of bias is 32(# of patch)
h_conv1=tf.nn.relu(conv2d(x_image,W_conv1)+b_conv1)  # y=aW+bias. relu:non-linear process. Output size 28x28x32
h_pool1=max_pool_2x2(h_conv1)  # Output side 14*14*32 because the strides is 2

#conv2 layer + Pooling########
W_conv2=weight_variable([5,5,32,64]) #patch side:5x5, 32:in side(unit per patch), 64:out side(depth)
b_conv2=bias_variable([64]) #the max length of bias is 64 (# of filter)
h_conv2=tf.nn.relu(conv2d(h_pool1,W_conv2)+b_conv2)  # y=aW+bias. relu:non-linear process. Output size 14x14x64
h_pool2=max_pool_2x2(h_conv2)  # Output side 7*7*64 because the strides is 2

#fully connected layer1#########
W_fc1=weight_variable([7*7*64,1024])#Input shape= size of conv2 output, Output size is 1024
b_fc1=bias_variable([1024])  # number of output

h_pool2_flat=tf.reshape(h_pool2,[-1,7*7*64]) #Flatten,[n_samples,7,7,64]==>[n_samples,7*7*64]
h_fc1=tf.nn.relu(tf.matmul(h_pool2_flat,W_fc1)+b_fc1)  #relu:activation function
h_fc1_drop=tf.nn.dropout(h_fc1,keep_prob) # fix overfitting
                     
#fully connected layer2#########
W_fc2=weight_variable([1024,10])#Input shape= 1024???????, Output size is 10(identify 0~9)
b_fc2=bias_variable([10])  
predition=tf.nn.softmax(tf.matmul(h_fc1_drop,W_fc2)+b_fc2) #softmax:activation function


                 

#==========================================================================================
#add hidden layers and predition layer

#hidden_layer1=add_layer(xs,1,10,activation_function=tf.nn.relu)

#hidden_layer2=add_layer(hidden_layer1,10,10,activation_function=tf.nn.relu)

#add output layer
#predition=add_layer(xs,784,10,activation_function=tf.nn.softmax)  #softmax generally for classfication 

#============================================================================================
#the error between predition and real data
cross_entropy=tf.reduce_mean(-tf.reduce_sum(ys*tf.log(predition),reduction_indices=[1])) #loss
#loss=tf.reduce_mean(tf.reduce_sum(tf.square(ys-predition),reduction_indices=[1]))  #loss function
# sum for all inputs then mean. reduction_indices????


#AdamOptimizer: Grandint Desense
#train_step=tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)  #back propogation, 0.1 is learning rate
train_step=tf.train.AdamOptimizer(1e-4).minimize(cross_entropy) 




sess=tf.Session()

#init=tf.initialize_all_variables()  #important step

sess.run(tf.global_variables_initializer())   #important step

#sess.run(init)

for i in range(1000):
    batch_xs,batch_ys=mnist.train.next_batch(100)  #Extract 100 examples as train data from xs,ys for training per time
    #x_image=np.reshape(batch_xs,[-1,28,28,1])
    print(np.shape(batch_xs))
    #print(batch_ys)
    print(np.shape(batch_ys))
    #print(np.shape(x_image))
    #training
    
    #sess.run(train_step,feed_dict={xs:batch_xs,ys:batch_ys})
    sess.run(train_step,feed_dict={xs:batch_xs,ys:batch_ys,keep_prob:0.5})
    
    if i % 50==0:
        #print(sess.run(loss,feed_dict={xs:x_data,ys:y_data}))
        #print(sess.run(predition,feed_dict={xs:x_data,ys:y_data}))
        print(compute_accuracy(mnist.test.images,mnist.test.labels))  #test means test data
        
        


Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
(100, 784)
(100, 10)
0.0932
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(

(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
0.946
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 7

(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
0.9655
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 784)
(100, 10)
(100, 